In [ ]:
#Library 
from bs4 import BeautifulSoup
from datetime import date
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
from selenium.webdriver.chrome.options import Options
import os
import shutil
import urllib.request
import pandas as pd
import PIL
from PIL import Image
from instabot import Bot

In [ ]:
def resize(image_pil):
    '''
    Resize PIL image keeping ratio and using white background.
    '''

    image_pil = (Image.open(image_pil, 'r')) #Usar a diretoria como stuff
    width = 1080
    height = 1080

    ratio_w = width / image_pil.width
    ratio_h = height / image_pil.height
    if ratio_w < ratio_h:
        # It must be fixed by width
        resize_width = width
        resize_height = round(ratio_w * image_pil.height)
    else:
        # Fixed by height
        resize_width = round(ratio_h * image_pil.width)
        resize_height = height
    image_resize = image_pil.resize((resize_width, resize_height), Image.ANTIALIAS)
    background = Image.new('RGBA', (width, height), (255, 255, 255, 255))
    offset = (round((width - resize_width) / 2), round((height - resize_height) / 2))
    background.paste(image_resize, offset)
    return background.convert('RGB')

In [ ]:
options = Options()
options.add_argument("--disable-notifications")
driver = webdriver.Chrome(executable_path='C:\\Users\\svnduw\\.wdm\\drivers\\chromedriver\\win32\\93.0.4577.15\\chromedriver.exe', chrome_options=options)

In [ ]:
import pandas as pd
folder=pd.read_excel(r'C:\Users\svnduw\Desktop\Bekbol\Bekbol project\Bekbol\Urls\20.10.2021.xlsx')

In [ ]:
urls=folder['urls'].tolist()

In [ ]:
name=[]
img_folder=[]
price=[]
link=[]
for i in range(len(urls)):
    try:
        driver.get(str(urls[i]))
        time.sleep(2)
       # pages=driver.find_elements_by_class_name('p-card-wrppr')
       # pages[i].click()   
                                      
        #main_page = driver.current_window_handle
        
        temp=[]
       # for handle in driver.window_handles:
           # if handle != main_page:
               # additional_page = handle
        #driver.switch_to.window(additional_page)
        link.append(driver.current_url)
        html = driver.page_source
        soup = BeautifulSoup(html,"html.parser")
        time.sleep(2)
        for tag in soup.find_all(class_="pr-new-br"):
            name.append(tag.text)        
        for tag in soup.find_all(class_="prc-slg") :            
            temp.append(tag.text)
        price.append(temp[0])
        #for tag in soup.find_all(class_="prc-slg") : 
            #price.append(tag.text)
        img=[]
        for a in soup.find_all(class_="product-slide"):            
            if a.img:
                img.append(a.img['src'])
        img_len=len(img)
        if img_len>0:
            for i in range(len(img)):
                img[i]='/'.join(img[i].split('/')[0:3])+'/'+'/'.join(img[i].split('/')[6:])
            for j in range(len(img)):
                urllib.request.urlretrieve(img[j], r'C:\Users\svnduw\Desktop\Bekbol\Bekbol project\Bekbol\Images\20.10.2021\\'+str(img[j].split('/')[10])+'_'+str(j)+'.jpg')
            img_folder.append(img[0].split('/')[10])
        elif(img_len==0):
            for a in soup.find_all(class_="base-product-image"):
                if a.img:
                    img.append(a.img['src'])
            for j in range(len(img)):
                urllib.request.urlretrieve(img[j], r'C:\Users\svnduw\Desktop\Bekbol\Bekbol project\Bekbol\Images\20.10.2021\\'+str(img[j].split('/')[10])+'_'+'.jpg')

            img_folder.append(img[0].split('/')[10])           
            
        #driver.close()        
        #driver.switch_to.window(main_page)        
    except ElementClickInterceptedException:        
        driver.refresh()
    except NoSuchElementException:          
        driver.refresh()  
driver.close()

In [ ]:
today = date.today()
category=['Часы']*len(name)
d1 = today.strftime("%d/%m/%Y")
d = {'Дата':d1, 'Категория':category, 'Название':name, 'Цена(Лира)':price, 'Cсылка на фото':img_folder,'Ссылка':link}
df=pd.DataFrame(d)
df['Цена(Лира)']=df['Цена(Лира)'].map(lambda x: x.rstrip(' TL'))
df=df.applymap(lambda x: str(x.replace(',','.')))
df['Цена(Лира)']=df['Цена(Лира)'].astype(float)

In [ ]:
df.at[6, 'Цена(Лира)'] = 300


In [ ]:
df

In [ ]:
df['Cсылка на фото'].nunique()

In [ ]:
driver = webdriver.Chrome(executable_path='C:\\Users\\svnduw\\.wdm\\drivers\\chromedriver\\win32\\93.0.4577.15\\chromedriver.exe')
driver.get("https://www.xe.com/currencyconverter/convert/?Amount=1&From=TRY&To=KZT")
time.sleep(2)
html = driver.page_source
soup = BeautifulSoup(html,"html.parser")
for tag in soup.find_all(class_="result__BigRate-sc-1bsijpp-1 iGrAod"):
    currency=tag.text.split(' ')[0]
driver.close()
currency=float(currency)

In [ ]:
currency

In [ ]:
df['Цена(kzt)']=(df['Цена(Лира)']*currency).round()
df['Доставка за 1 кг']=6000
df['Цена на Instagram']=(df['Цена(kzt)']*2+df['Доставка за 1 кг']).astype(int).round(-2)
df['Profit']=(df['Цена на Instagram']-(df['Цена(kzt)']+df['Доставка за 1 кг'])).astype(int)

In [ ]:
df

In [ ]:
photo_list=df['Cсылка на фото'].tolist()
name_list=df['Название'].tolist()
price_list=df['Цена на Instagram'].tolist()

In [ ]:
path=r'C:\Users\svnduw\Desktop\Bekbol\Bekbol project\Bekbol\Images\20.10.2021'
new_path=r'C:\Users\svnduw\Desktop\Bekbol\Bekbol project\Bekbol\Resized\20.10.2021'
files=os.listdir(path)
for file in files:
    resize(path+'\\'+file).save(new_path+'\\'+file)

In [ ]:
import os
path=r'C:\Users\svnduw\Desktop\Bekbol\Bekbol project\Bekbol\Resized\20.10.2021'
files = os.listdir(path)
from instagrapi import Client
Username='qyran_watches'
Password='****'
bot = Client()
bot.login(Username, Password)
for j in range(len(photo_list)):
    new=[]
    for i in files:
        if photo_list[j]==i.split('_')[0]:
            new.append('C:\\Users\\svnduw\\Desktop\\Bekbol\\Bekbol project\\Bekbol\\Resized\\20.10.2021\\'+str(i))
            time.sleep(2)
    img_leng=len(new)
    if img_leng!=1:
        text = '⌚ '+str(name_list[j].split(' ')[0])+'\n 🔴 Производство: Турция \n 💵 Цена: '+str(price_list[j])+'\n ♻️ Для заказа можете написать нам по прямой ссылке в шапке профиля \n 📲 8 707 946 6496 \n 💸Предоплата 50% \n_____________________ \n Доставка по всему РК\n_ _ _ _ _ _ _ _ _ _ _ _ _\n Наши магазины:\n @ant_wears - интернет-магазин женской (мусульманской) одежды\n @gstyle_lady - интернет-магазин женской одежды\n @social_shoe_store - интернет-магазин обуви\n @aishoes.kz - интернет-магазин женской обуви\n @ayalin_kids - интернет-магазин детской одежды\n @ayamin_bags - интернет-магазин люксовых сумок'
        bot.album_upload(
        new,
        caption = text)
    elif img_leng==1:
        text = '⌚ '+str(name_list[j].split(' ')[0])+'\n 🔴 Производство: Турция \n 💵 Цена: '+str(price_list[j])+'\n ♻️ Для заказа можете написать нам по прямой ссылке в шапке профиля \n 📲 8 707 946 6496 \n 💸Предоплата 50% \n_____________________ \n Доставка по всему РК\n_ _ _ _ _ _ _ _ _ _ _ _ _\n Наши магазины:\n @ant_wears - интернет-магазин женской (мусульманской) одежды\n @gstyle_lady - интернет-магазин женской одежды\n @social_shoe_store - интернет-магазин обуви\n @aishoes.kz - интернет-магазин женской обуви\n @ayalin_kids - интернет-магазин детской одежды\n @ayamin_bags - интернет-магазин люксовых сумок'
        bot.photo_upload(new[0],caption=text)

In [ ]:
excelfilename = today.strftime("%B %d,%Y") +".xlsx"
df.to_excel(r'C:\\Users\\svnduw\\Desktop\\Bekbol\\Bekbol project\\Bekbol\\Report\\'+excelfilename)  

In [ ]:
from instagrapi import Client
from instagrapi.story import StoryBuilder, StoryMention
path=r'C:\Users\svnduw\Desktop\Bekbol\Bekbol project\Bekbol\Resized\20.10.2021'
files = os.listdir(path)
adw0rd = bot.user_info_by_username('qyran_watches')
for i in files:
    bot.photo_upload_to_story(path+'\\'+i,
                             "@qyran_watches",
                             mentions=[StoryMention(user=adw0rd, x=0.49892962, y=0.703125, width=0.8333333333333334, height=0.125)])
